## Data Preprocessing

In [ ]:
#Terminal Commands (mostly for installing libraries we don't have)
%pip install netCDF4
%pip install requests
%pip install lxml
%pip install bs4

In [ ]:
#Import Statements

#Machine Learning
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

#Data
import netCDF4 as nc
import os
import requests
import lxml
import bs4
import pandas as pd
import tarfile

#Data Visualization
import matplotlib.pyplot as plt

#Upload to Google Drive
import shutil

In [ ]:
#Data Acquisition & Cleaning for Hurdat 2 Data

#Name of Hurriscan Data Folder
folderName = "HurriScan_Data"

#First check if the user already has the required data (so we do not have to download multiple times)
os.chdir("/content")
currDir = os.listdir()

print(currDir)

if (not(folderName in currDir)):
  #First let's get the Hurdat Data
  res = requests.get("https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2022-050423.txt")

  soup = bs4.BeautifulSoup(res.text, "lxml")

  #Write the soup to a file
  path = os.curdir + "/" + folderName
  os.mkdir(path)
  os.chdir(path)

  #Write text file of Hurdat 2 data (6.6 MB)
  f = open('Hurdat.txt', "w+")
  f.write(soup.text)
  f.close

  #Now, let us read this data into a database (table)
  #For this, we will be adapting the code from https://github.com/metemaad/HURDAT2_processor/blob/master/HURDAT2_processor.py

Record_identifier_dic={
'C':"Closest approach to a coast, not followed by a landfall"
,'G':"Genesis"
,'I':"An intensity peak in terms of both pressure and wind"
,'L':"Landfall (center of system crossing a coastline)"
,'P':"Minimum in central pressure"
,'R':"Provides additional detail on the intensity of the cyclone when rapid changes are underway"
,'S':"Change of status of the system"
,'T':"Provides additional detail on the track (position) of the cyclone"
,'W':"Maximum sustained wind speed"}


Status_of_system_dic={
'TD':"Tropical cyclone of tropical depression intensity (< 34 knots)"
,'TS':"Tropical cyclone of tropical storm intensity (34-63 knots)"
,'HU':"Tropical cyclone of hurricane intensity (> 64 knots)"
,'EX':"Extratropical cyclone (of any intensity)"
,'SD':"Subtropical cyclone of subtropical depression intensity (< 34 knots)"
,'SS':"Subtropical cyclone of subtropical storm intensity (> 34 knots)"
,'LO':"A low that is neither a tropical cyclone, a subtropical cyclone, nor an extratropical cyclone (of any intensity)"
,'WV':"Tropical Wave (of any intensity)"
,'DB':"Disturbance (of any intensity)"}

def process_details(data):
    data=data.split(',')
    Year=int(data[0][0:4])
    Month=int(data[0][4:6])
    Day=int(data[0][6:8])
    Hours_in_UTC=int(data[1][0:2])
    Minutes_in_UTC=int(data[1][2:4])
    Record_identifier=data[2].strip()
    try:
        Record_identifier_desc=Record_identifier_dic[data[2].strip()]
    except:
        Record_identifier_desc=None

    Status_of_system=data[3].strip()
    try:
        Status_of_system_desc=Status_of_system_dic[Status_of_system]
    except:
        Status_of_system_desc=None

    if data[4].strip()[-1:] in ('N','S'):
        if data[4].strip()[-1:]=='N':
            Latitude=float(data[4].strip()[:-1])
        else:
            Latitude=-1.0*float(data[4].strip()[:-1])
    else:
        Latitude=-999

    if data[5].strip()[-1:] in ('E','W'):
        if data[5].strip()[-1:]=='E':
            Longitude=float(data[5].strip()[:-1])
        else:
            Longitude=-1.0*float(data[5].strip()[:-1])
    else:
        Longitude=-999
    Maximum_sustained_wind_in_knots=float(data[6].strip())
    Minimum_Pressure_in_millibars=float(data[7].strip())
    i=8
    F34_kt_wind_radii_maximum_northeastern=float(data[i].strip())
    i+=1
    F34_kt_wind_radii_maximum_southeastern=float(data[i].strip())
    i+=1
    F34_kt_wind_radii_maximum_southwestern=float(data[i].strip())
    i+=1
    F34_kt_wind_radii_maximum_northwestern=float(data[i].strip())


    i+=1
    F50_kt_wind_radii_maximum_northeastern=float(data[i].strip())
    i+=1
    F50_kt_wind_radii_maximum_southeastern=float(data[i].strip())
    i+=1
    F50_kt_wind_radii_maximum_southwestern=float(data[i].strip())
    i+=1
    F50_kt_wind_radii_maximum_northwestern=float(data[i].strip())

    i+=1
    F64_kt_wind_radii_maximum_northeastern=float(data[i].strip())
    i+=1
    F64_kt_wind_radii_maximum_southeastern=float(data[i].strip())
    i+=1
    F64_kt_wind_radii_maximum_southwestern=float(data[i].strip())
    i+=1
    F64_kt_wind_radii_maximum_northwestern=float(data[i].strip())



    res=Year,Month,Day,Hours_in_UTC,Minutes_in_UTC,Record_identifier,Record_identifier_desc,Status_of_system,Status_of_system_desc,Latitude,Longitude,Maximum_sustained_wind_in_knots,Minimum_Pressure_in_millibars,F34_kt_wind_radii_maximum_northeastern,F34_kt_wind_radii_maximum_southeastern,F34_kt_wind_radii_maximum_southwestern,F34_kt_wind_radii_maximum_northwestern,F50_kt_wind_radii_maximum_northeastern,F50_kt_wind_radii_maximum_southeastern,F50_kt_wind_radii_maximum_southwestern,F50_kt_wind_radii_maximum_northwestern,F64_kt_wind_radii_maximum_northeastern,F64_kt_wind_radii_maximum_southeastern,F64_kt_wind_radii_maximum_southwestern,F64_kt_wind_radii_maximum_northwestern
    return res

def process_header(data):
    data=data.split(',')
    Basin,ATCF_cyclone_number_for_that_year,Year,Name,Number_of_best_track_entries=data[0][0:2],data[0][2:4],data[0][4:8],data[1].strip(),data[2].strip()
    res=Basin,ATCF_cyclone_number_for_that_year,Year,Name,Number_of_best_track_entries
    return res


def identify_line_type(data):
    #print(data.split(',')) #Commented out because printing is slow and has lots of output
    if len(data.split(','))>4:
        return 2
    else:
        return 1
def columns_name():
    res=['Basin','ATCF_cyclone_number_for_that_year','Year_','Name',
         #'Number_of_best_track_entries',
         'Year','Month','Day','Hours_in_UTC','Minutes_in_UTC',
         'Record_identifier','Record_identifier_desc','Status_of_system','Status_of_system_desc','Latitude','Longitude'
         ,'Maximum_sustained_wind_in_knots','Minimum_Pressure_in_millibars','F34_kt_wind_radii_maximum_northeastern',
         'F34_kt_wind_radii_maximum_southeastern','F34_kt_wind_radii_maximum_southwestern',
         'F34_kt_wind_radii_maximum_northwestern','F50_kt_wind_radii_maximum_northeastern',
         'F50_kt_wind_radii_maximum_southeastern','F50_kt_wind_radii_maximum_southwestern',
         'F50_kt_wind_radii_maximum_northwestern','F64_kt_wind_radii_maximum_northeastern',
         'F64_kt_wind_radii_maximum_southeastern','F64_kt_wind_radii_maximum_southwestern',
         'F64_kt_wind_radii_maximum_northwestern']
    return res

pf=[]
header_fields=[]
filepath = '/content/HurriScan_Data/Hurdat.txt'
with open(filepath) as fp:
    ln = fp.readline()
    while ln:

        lt=identify_line_type(ln)

        details=[]
        if (lt==1):
            header_fields=process_header(ln)
            details=[]
        else:
            details=process_details(ln)
        if (details!=[]):
            n=list(header_fields[:-1])+list(details)

            pf.append(n)
        ln=fp.readline()

df=pd.DataFrame(pf)
df.columns=columns_name()
df.to_csv('hur.csv')
print(f'Number of samples before cleaning is: {df.shape[0]}')

#At this point, we have a dataframe with all of the Hurdat 2 data.
#Now let us clean it so it only has information about name, time and maximum windspeed

#Contains the columns we want to remove
columnList = list(df.columns)

#Remove columns we want to keep!
columnList.remove("Name")
columnList.remove("Year")
columnList.remove("Month")
columnList.remove("Day")
columnList.remove("Hours_in_UTC")
columnList.remove("Minutes_in_UTC")
columnList.remove("Maximum_sustained_wind_in_knots")
columnList.remove("Basin")

#Drop irrelevant columns
df = df.drop(columns = columnList)

#If it does not have a name, drop it from data
df = df[df['Name'] != 'UNNAMED']
df = df[df['Year'] > 1978]
df = df[df['Year'] < 2016]
#df = df[df['Maximum_sustained_wind_in_knots'] > 63]


#Add the Saffir Simpson (intensity rating, to our dataframe)
df['Saffir_Simpson_Rating'] = ''

#Add column for satellite image of hurricane
df['Satellite_Image'] = ''

rowNum = 0

#Update Saffir Simpson rating column based on maximum sustained windspeed column
for index, row in df.iterrows():
  if row['Maximum_sustained_wind_in_knots'] in range(64, 83):
    df['Saffir_Simpson_Rating'].values[rowNum] = 1
  elif row['Maximum_sustained_wind_in_knots'] in range(83, 96):
    df['Saffir_Simpson_Rating'].values[rowNum] = 2
  elif row['Maximum_sustained_wind_in_knots'] in range(96, 113):
    df['Saffir_Simpson_Rating'].values[rowNum] = 3
  elif row['Maximum_sustained_wind_in_knots'] in range(113, 137):
    df['Saffir_Simpson_Rating'].values[rowNum] = 4
  elif row['Maximum_sustained_wind_in_knots'] > 137:
    df['Saffir_Simpson_Rating'].values[rowNum] = 5
  else:
    df['Saffir_Simpson_Rating'].values[rowNum] = 0

  rowNum += 1

#Print out first couple of element to make sure we have done it correctly
print(f'Number of samples after cleaning is: {df.shape[0]}')
df.reset_index(inplace=True)
df.head()

#Wow! Now we have fully cleaned the Hurdat 2 data

['.config', 'Images.npy', 'drive', 'hur.csv', 'HurriScan_Data', 'HurriScanData.npy', 'sample_data']
Number of samples before cleaning is: 53976
Number of samples after cleaning is: 14455


,index,Basin,Name,Year,Month,Day,Hours_in_UTC,Minutes_in_UTC,Maximum_sustained_wind_in_knots,Saffir_Simpson_Rating,Satellite_Image
0,33745,AL,ANA,1979,6,19,1,20,25.0,0,
1,33746,AL,ANA,1979,6,19,1,80,25.0,0,
2,33747,AL,ANA,1979,6,20,0,0,25.0,0,
3,33748,AL,ANA,1979,6,20,0,60,25.0,0,
4,33749,AL,ANA,1979,6,20,1,20,25.0,0,


In [ ]:
#Data Acquisition and Cleaning for Hursat Data

imagesList = []

#First, check to see if we already have the data
print(os.getcwd())
os.chdir("/content/HurriScan_Data")

#If we have not already read the data
if not("Hursat" in os.listdir()):

  #Make the Hursat folder
  os.mkdir("Hursat")
  os.chdir("/content/HurriScan_Data/Hursat")

  #Initialize to starting year
  year = 2015
  numImages = 0

  #Keep adding data until we have 12000 images that match
  while year > 1977 and numImages < 12000:
    url = 'https://www.ncei.noaa.gov/data/hurricane-satellite-hursat-b1/archive/v06/' + str(year)

    #Get the webpage for the current year in Hursat data
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text, "html.parser")

    #This line is from https://github.com/23ccozad/hurricane-wind-speed-cnn/blob/master/download.py to extract download url
    year_directory_file_urls = [url + '/' + node.get('href') for node in
                                      soup.find_all('a') if node.get('href').endswith('tar.gz')]

    #Iterate through all the .tar file urls
    for storm_url in year_directory_file_urls:
      #Now we need to check if this storm is in the Hurdat 2 data
      nameStorm = storm_url.split('_')[-2]
      yearStorm = int(storm_url.split('_')[3][:4])

      print(f'Storm Name: {nameStorm} and Year of Storm: {yearStorm}')
      print(f"Link: {storm_url}")

      #If the name is not in our Hurdat 2 dataset, we can skip as it will not be useful
      if not(nameStorm in df['Name'].values):
        print("Skipping as it is not in our Hurdat data")
        continue

      print("Found Match of storm name and year!")

      #Now we know that this storm is in the Hurdat data, we can read it's file to see if further information matches

      #First let us download the storm data from online
      # Open the .tar.gz and copy it's contents from the web, onto our computer
      path = os.getcwd() + '/' + nameStorm

      #Tar file and NetCDF processing code from https://github.com/23ccozad/hurricane-wind-speed-cnn/blob/master/download.py
      request = requests.get(storm_url, allow_redirects=True)
      open(nameStorm, 'wb').write(request.content)
      request.close()


      #response = requests.get(storm_url, stream=True)
      #if response.status_code == 200:
          #with open(path, 'wb') as f:
              #f.write(response.raw.read())
      #else:
        #print("error getting tar file")


      tar = tarfile.open(path)
      file_prefixes_in_directory = []
      for file_name in tar.getnames():

        #print(f"File Name: {file_name}")

        # Get the date and time of the satellite image, and the name of the satellite that took the image
        fulldate = file_name.split(".")[2] + file_name.split(".")[3] + file_name.split(".")[4]
        time = file_name.split(".")[5]

        # Determine whether the best track dataset has a record for the date and time of this storm.

        #print(f'Full Date: {fulldate} and time: {time}')
        #The types are strings of fulldate and time

        #Convert Hursat date format to Hurdat date format

        hursatYear = int(fulldate[0:4])

        if (fulldate[4] == '0'):
          hursatMonth = int(fulldate[5])
        else:
          hursatMonth = int(fulldate[4:6])

        if (fulldate[6] == '0'):
          hursatDay = int(fulldate[7])
        else:
          hursatDay = int(fulldate[6:])

        hursatTime = time[0:2]

        if (hursatTime[0] == '0'):
          hursatTime = int(hursatTime[1])
        else:
          hursatTime = int(hursatTime)


        #Check if the date and time is in the Hurdat 2 data


        #print(f"hursatYear: {hursatYear}, hursatMonth: {hursatMonth}, hursatDay: {hursatDay}, hursatTime: {hursatTime}")

        index_list = df[(df['Year'] == hursatYear)&(df['Month'] == hursatMonth)&(df['Day'] == hursatDay) & (df['Hours_in_UTC'] == hursatTime)].index.tolist()

        #print("Index List:")
        #print(index_list)

        #If the list of indices we received was empty, we did not find a match in Hurdat
        if (len(index_list) == 0):
          print("Did not find exact match in Hurdat Data Rows")
        #We found a match in Hurdat
        else:
          print("Found a match in Hurdat data")
          try:
              tar.extract(file_name, '/content/HurriScan_Data/Hursat')
          except KeyError:
              print(f"Warning: File '{file_name}' not found in the tar archive.")
              continue

          #Open the netCDF file and extract the image

          filePath = '/content/HurriScan_Data/Hursat/' + file_name

          dSet = nc.Dataset(filePath)
          #df['Satellite_Image'].values[index_list[0]] = dSet.variables['IRWIN'][0]
          #df.at['Satellite_Image', int(index_list[0])] = numImages
          print("Before: ")
          print(df.at[index_list[0], 'Satellite_Image'])
          #df.at[int(index_list[0]), 'Satellite_Image'] = numImages
          #df['Satellite_Image'][int(index_list[0])] = numImages
          imagesList.append(dSet.variables['IRWIN'][0])
          df.at[index_list[0], 'Satellite_Image'] = len(imagesList) - 1
          #numpArray[index_list[0]][9] = numImages
          print("After: ")
          print(df.at[index_list[0], 'Satellite_Image'])
          numImages += 1
      tar.close()
      print(" ***** DONE WITH THIS STORM **** \n\n")

    year -= 1


Streaming output truncated to the last 5000 lines.
Found a match in Hurdat data
Before: 
6766
After: 
6767
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Found a match in Hurdat data
Before: 

After: 
6768
Found a match in Hurdat data
Before: 
6768
After: 
6769
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact match in Hurdat Data Rows
Did not find exact m

In [ ]:
#Get rid of files for easy testing
os.chdir("/content/HurriScan_Data/Hursat/")
files = list(os.listdir())
#files.remove(".ipynb_checkpoints")
print(files)

for file in files:
  toRemove = "/content/HurriScan_Data/Hursat/" + file
  os.remove(toRemove)

os.chdir("/content/HurriScan_Data")
os.rmdir("Hursat")

['2003284N19291.MINDY.2003.10.12.0000.28.GOE-12.031.hursat-b1.v06.nc', '2000217N11342.ALBERTO.2000.08.04.0000.27.MET-7.026.hursat-b1.v06.nc', '1979197N13314.CLAUDETTE.1979.07.21.0000.66.GOE-3.018.hursat-b1.v06.nc', '2003286N10323.NICHOLAS.2003.10.26.0000.65.MET-7.018.hursat-b1.v06.nc', '2000231N14308.CHRIS.2000.08.19.0000.29.GOE-8.022.hursat-b1.v06.nc', '1997246N11316.ERIKA.1997.09.16.0000.57.MET-6.044.hursat-b1.v06.nc', '1993217N10320.BRET.1993.08.09.0000.47.GOE-7.035.hursat-b1.v06.nc', '1981250N15306.GERT.1981.09.10.0000.74.GOE-4.035.hursat-b1.v06.nc', '2001157N28265.ALLISON.2001.06.15.0000.40.GOE-8.022.hursat-b1.v06.nc', '1986226N30276.CHARLEY.1986.08.14.0000.40.GOE-6.009.hursat-b1.v06.nc', '1984348N35300.LILI.1984.12.16.0000.68.MET-2.048.hursat-b1.v06.nc', '1995210N26273.DEAN.1995.07.31.0000.41.GOE-8.035.hursat-b1.v06.nc', '2012242N13333.LESLIE.2012.09.10.0000.42.GOE-13.048.hursat-b1.v06.nc', '1985299N25270.JUAN.1985.10.28.0000.34.GOE-6.057.hursat-b1.v06.nc', '2001327N31309.OLGA.20

In [ ]:
# create figure
#fig = plt.figure(figsize=(301, 301))

# loop over images
#for i in range(len(imagesList)):
    #img = np.reshape(imagesList[i : (i + 1)], (301, 301))
    #fig.add_subplot(len(imagesList)//2, len(imagesList)//2, i + 1)
    #plt.imshow(img)

#for image in imagesList:
  #plt.imshow(image)

In [ ]:
df.head(1000)

,index,Basin,Name,Year,Month,Day,Hours_in_UTC,Minutes_in_UTC,Maximum_sustained_wind_in_knots,Saffir_Simpson_Rating,Satellite_Image
0,33745,AL,ANA,1979,6,19,1,20,25.0,0,
1,33746,AL,ANA,1979,6,19,1,80,25.0,0,
2,33747,AL,ANA,1979,6,20,0,0,25.0,0,
3,33748,AL,ANA,1979,6,20,0,60,25.0,0,
4,33749,AL,ANA,1979,6,20,1,20,25.0,0,
...,...,...,...,...,...,...,...,...,...,...,...
995,35208,AL,ALBERTO,1982,6,3,0,0,30.0,0,
996,35209,AL,ALBERTO,1982,6,3,0,60,40.0,0,
997,35210,AL,ALBERTO,1982,6,3,1,20,50.0,0,
998,35211,AL,ALBERTO,1982,6,3,1,80,75.0,1,


In [ ]:
len(imagesList)

7100

In [ ]:
#Get rid of any rows in the dataframe that don't correpond to images
df['Satellite_Image'].replace('', np.nan, inplace=True)
df.dropna(subset=['Satellite_Image'], inplace=True)

#Drop irrelevant columns in dataframe (drop all except rating and index of image)
columnList = list(df.columns)
columnList.remove("Satellite_Image")
columnList.remove("Saffir_Simpson_Rating")
df = df.drop(columns = columnList)
df.head(10)

ratings_list = df['Saffir_Simpson_Rating'].tolist()
image_indices = df['Satellite_Image'].tolist()
images = []
for index in image_indices:
  images.append(imagesList[int(index)])

#Sanity check!
#plt.imshow(images[0])
#plt.imshow(imagesList[6971])

In [ ]:
#Convert Image data List to a numpy array
imageData = np.array(images)
intensity_ratings = np.array(ratings_list)

#The number of training images we have
print(f"The number of labels we have to train with is: {intensity_ratings.shape[0]}")
print(f"The number of images we have to train with is: {len(imageData)}")

#Save our numpy arrays so that they may easily be loaded in the future
np.save('/content/Images.npy', imageData)
np.save('/content/IntensityRatings.npy', intensity_ratings)


NameError: ignored

In [ ]:
#Mount Google Drive and save .npy files to them
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

#Please replace the variable with your custom path when you want to run the notebook
claire = 'APS360_Team_11_Project_Folder'
sean = 'University/Third_Year/APS360/APS360_Project'
charlotte = ''
thardchi = ''

#Save Images and Labels to Shared Google Drive
shutil.copy("/content/Images.npy",f"/content/drive/MyDrive/{claire}/HurriScan_Data")
shutil.copy("/content/IntensityRatings.npy",f"/content/drive/MyDrive/{claire}/HurriScan_Data")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/APS360_Team_11_Project_Folder/HurriScan_Data/IntensityRatings.npy'